In [ ]:
from cdcqr.data.glassnode import glassnode_data as gnd
from cdcqr.ct.utils import plot2
import pandas as pd
import cdcqr.analytics.utils as autil 
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from cdcqr.analytics.utils import calculate_corr_pvalues
from cdcqr.common.utils import LOCAL_DATA_DIR
import os
import numpy as np
from cdcqr.backtest.vbt.runsignal import runsignal
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import time
from IPython.display import Audio
sound_file = '/core/tmp/ding.wav'
%matplotlib inline

import sys
sys.path.append('/core/github/cryptoderiv-quant/')
from ct.utils import pv

C:\Users\Wang\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
gn = gnd.GlassnodeData()

In [ ]:
gn.get_feature_best_resolutions('balance_exchanges')

In [ ]:
fs = ['price_usd_close', 'balance_exchanges']
assets = ['BTC','USDT', 'USDC', 'BUSD']
df = gn.load_features(fs, assets, resolution='10m')

In [ ]:
df1 = df[df.index>='20200101'].ffill().drop(['USDT-price_usd_close','USDC-price_usd_close','BUSD-price_usd_close'],axis=1)
df1['stable-balance_exchanges'] = df['USDT-balance_exchanges']+df['USDC-balance_exchanges']+df['BUSD-balance_exchanges']
df1 = df1[['BTC-price_usd_close', 'stable-balance_exchanges','BTC-balance_exchanges']]
df1.head()

In [ ]:
df1['BTC-balance_exchanges_MC'] = df1['BTC-price_usd_close'] * df1['BTC-balance_exchanges']

In [ ]:
df1['BTCSSR'] = df1['BTC-balance_exchanges_MC']/df1['stable-balance_exchanges']
df2 = df1.drop('BTC-balance_exchanges_MC', axis=1).dropna()

In [ ]:
df3 = df2.copy()
df3['ret'] = df3['BTC-price_usd_close'].pct_change()
df3['r+5'] = df3['BTC-price_usd_close'].pct_change(periods=5).shift(-5)
df3['r+10'] = df3['BTC-price_usd_close'].pct_change(periods=10).shift(-10)
df3['r+100'] = df3['BTC-price_usd_close'].pct_change(periods=100).shift(-100)

df3['stable_bal_chg'] = df3['stable-balance_exchanges'].diff()
df3['stable_bal_chg_lag1'] = df3['stable_bal_chg'].shift(1)
df3['BTCSSR_lag1'] = df3['BTCSSR'].shift(1)
df3['BTCSSR_chg'] = df3['BTCSSR'].diff()
df3['BTCSSR_chg_lag1'] = df3['BTCSSR_chg'].shift(1)

In [ ]:
df4 = df3.dropna()[['stable_bal_chg', 'stable_bal_chg_lag1', 'BTCSSR', 'BTCSSR_chg', 'BTCSSR_chg_lag1', 'ret', 'r+5', 'r+10', 'r+100']]

In [ ]:
df4a = df4.where(~np.isinf(df4), 0)

In [ ]:
lookback = 4000
t0 = time.time()
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
df4a['BTCSSR_pctrank4k'] = df4a['BTCSSR'].rolling(lookback).apply(pctrank)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
df4b = df4a[df4a.index>='20210101']

In [ ]:
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))

In [ ]:
df = pd.read_pickle('/core/tmp/long-BTCSSR_pctrank4k_res_df.pickle')

In [ ]:
df.pv()

In [ ]:
feature = 'BTCSSR_pctrank4k'
p=price.resample('1Min').last().ffill()

In [ ]:
tp=15
sl=5
n1=None
n2=None
qtl=0.995
short=False
signal = df4b[feature]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
signal = signal.reindex(p.index).ffill()

ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])

In [ ]:
runsignal?

In [ ]:
ressig['ma_tpsl_pf'].plot(subplots=['drawdowns','trade_pnl','cum_returns'])

In [ ]:
t0 = time.time()
feature = 'BTCSSR_pctrank4k'
p=price.resample('1Min').last().ffill()
tp=7
sl=5
pf = 'rsi_tpsl_pf'
n1=None
n2=None
qtl=0.01
short=True
signal = df4b[feature]<qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
signal = signal.reindex(p.index).ffill()

ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,
                 rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])

print(time.time()-t0)

In [8]:
import pandas as pd
import os
from cdcqr.common.utils import LOCAL_DATA_DIR
import sys
sys.path.append('/core/github/cryptoderiv-quant/')
from ct.utils import pv


logging to C:\core\logs\ct.log


In [ ]:
ressig[pf].plot(subplots=['drawdowns','trade_pnl','cum_returns'])

### load res_df files

In [6]:
res_df_fname1 = 'BTC_long-BTCPERPFundingRate_4997d75.pickle'
df1 = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'backtest_res', res_df_fname1))
df1.shape

(430080, 18)

In [9]:
df1.pv()